<!--HEADER-->
*[Guia de aulas da disciplina de Modelagem Matemática](https://github.com/rmsrosa/modelagem_matematica) do [IM-UFRJ](https://www.im.ufrj.br).*

<!--NAVIGATOR-->

<a href="https://colab.research.google.com/github/rmsrosa/modelagem_matematica/blob/master/aulas/12.05-Projeto3.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
&nbsp;
<a href="https://mybinder.org/v2/gh/rmsrosa/modelagem_matematica/master?filepath=aulas/12.05-Projeto3.ipynb"><img align="left" src="https://mybinder.org/badge.svg" alt="Open in binder" title="Open and Execute in Binder"></a>
&nbsp;
<a href="https://rmsrosa.github.io/modelagem_matematica/aulas/12.05-Projeto3.slides.html"><img align="left" src="https://rmsrosa.github.io/jupyterbookmaker/badges/slides_badge.svg" alt="Open slides" title="Open and View Slides"></a>
&nbsp;

[<- Aula 12: Epidemiologia - outros modelos compartimentais](12.00-Aula12.ipynb) | [Página Inicial](00-Sumario.ipynb) | [Aula 13: Epidemiologia: o modelo SIR em rede ->](13.00-Aula13.ipynb)

---


# Projeto 3: Controle de epidemia via vacinação

## Objetivos

- Ajustar os parâmtros do modelo SIR aos dados de uma epidemia de gripe
- Escolher uma melhor estratégia de vacinação para o controle de futuras epidemias

## Importando bibliotecas e definindo funções a serem usadas abaixo

In [1]:
from os import path

import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
from scipy.integrate import odeint

import networkx as nx

import matplotlib.pyplot as plt

import io, base64
from IPython.display import Image, HTML

## Modelo SIR de epidemia sem vitalidade e sem vacinação

O modelo SIR clássico pode ser representado pela série de processos

$$ \rm S \stackrel{\beta I/N}{\rightarrow} \rm I \stackrel{\gamma}{\rightarrow} \rm R,
$$

onde 
- $S$ é a **população de indivíduos suscetíveis**; 
- $I$ é a **população de indivíduos infectados**;
- $R$ é a **população de indivíduos recuparados**;
- $N=S+I+R$ é a **população total**;
- $\beta$ é um **fator de infecção**; e
- $\gamma$ é um **fator de recuperação**

### Sistema de equações diferenciais do modelo SIR clássico

Sistema:

$$ 
  \begin{cases}
    \displaystyle \frac{\rm d S}{\rm d t} =  - \beta \frac{I}{N}S, \\
    \displaystyle \frac{\rm d I}{\rm d t} =  \beta \frac{I}{N}S - \gamma I, \\
    \displaystyle \frac{\rm d R}{\rm d t} = \gamma I.
  \end{cases}
$$

## Modelo SIR com vitalidade, população constante e sem vacinação

Nesse caso, temos os processos

$$ \begin{cases}
     \rm N \stackrel{\nu}{\rightarrow} \rm S \stackrel{\beta I/N}{\rightarrow} \rm I \stackrel{\gamma}{\rightarrow} \rm R, & \text{(natalidade, infecção, recuperação)} \\
     \rm S \stackrel{\nu}{\rightarrow} M, & \text{(mortalidade entre os suscetíveis)} \\ 
     \rm I \stackrel{\nu}{\rightarrow} M, & \text{(mortalidade entre os infectados)} \\
     \rm R \stackrel{\nu}{\rightarrow} M, & \text{(mortalidade entre os recuperados)}
   \end{cases}
$$

onde $\nu$ indica tanto a taxa de natalidade como a de mortalidade. A taxa mundial de natalidade anual é da ordem de 18,5 por 1000 habitantes, ou seja 1,85%. Considerando a unidade de tempo como sendo de um dia, obtemos uma taxa diária da ordem de $\nu = 18,5/1000/365.25= 5.07 \times 10^{-5}$ (recém-nascidos por habitante por dia).

No entanto, ao considerarmos apenas uma região, digamos, a cidade do Rio de Janeiro, o parâmetro $\nu$ pode ser usado para indicar uma taxa de migração e ser de uma ordem significativamente maior do que a estimada acima.



## Modelo SIR com vitalidade e vacinação

Há vários modelos incluindo estratégias de vacinação. O modelo a seguir é discutido em G. Zaman, Y. H. Kang, I. H. Jung, "Stability analysis and optimal vaccination of an SIR epidemic model"; *Biosystems*, 93 (2008), 240-249, com o objetivo se obter um "controle ótimo", ou seja, a melhor estratégia de vacinação que minimize uma certa função custo.

Neste caso, a população de indivíduos suscetíveis, em um modelo SIR com vitalidade, é imunizada a uma taxa específica $u$, levando a um sistema de processos

$$ \begin{cases}
     \rm N \stackrel{\nu}{\rightarrow} \rm S \stackrel{\beta I/N}{\rightarrow} \rm I \stackrel{\gamma}{\rightarrow} \rm R, & \text{(natalidade, infecção, recuperação)} \\
     \rm S \stackrel{\nu}{\rightarrow} M, & \text{(mortalidade entre os suscetíveis)} \\ 
     \rm I \stackrel{\nu}{\rightarrow} M, & \text{(mortalidade entre os infectados)} \\
     \rm R \stackrel{\nu}{\rightarrow} M, & \text{(mortalidade entre os recuperados)} \\
     \rm S \stackrel{u}{\rightarrow} R, & \text{(imunização por vacinação dos suscetíveis)}
   \end{cases}
$$

onde $M$ é o número de **óbitos**, que é constante, visto que a taxa de mortalidade é igual à taxa de natalidade $\nu$.

### Sistema de equações diferenciais do modelo SIR com vitalidade e vacinação

Sistema:

$$ 
  \begin{cases}
    \displaystyle \frac{\rm d S}{\rm d t} =  \nu N - \beta \frac{I}{N}S - u S - \nu S, \\
    \displaystyle \frac{\rm d I}{\rm d t} =  \beta \frac{I}{N}S - \gamma I - \nu I, \\
    \displaystyle \frac{\rm d R}{\rm d t} = \gamma I + u S - \nu R
  \end{cases}
$$

### Otimização da processo de vacinação

Considerando uma doença não muito grave, que não cause óbitos, e onde o principal dano para a população como um todo seja a perda de produtividade, pode-se tentar estimar esse custo e equilibrar com o custo de vacinação para implementar uma política de vacinação que minimize o custo total. Isso nos leva a um problema de minimização de um **funcional-custo**.

O funcional-custo proposto em Zaman, Kang & Jung (2008) tem a forma

$$ J(u) = \int_0^T \left( A_1 S(t) + A_2 I(t) + \frac{\tau}{2}u(t)^2\right) \;\rm dt,
$$
onde $A_1$ e $A_2$ são parâmetros de "custo por indivíduo", das populações de suscetíveis e infectados, por conta de medidas de prevenção, no caso de indivíduos suscetíveis, e da perda de produtividade pela saída temporária do sistema produtivo, no caso de indivíduos infectados. O parâmetro $\tau$ é um fator de custo da vacinação.

O objetivo é encontrar uma função $u=u(t)$ que minimize $J(u)$, i.e.

$$ \min_{u\in \mathcal U} J(u),
$$

onde $\mathcal U$ é um **espaço de funções admissíveis**.

## O espaço de funções admissíveis

No artigo Zaman, Kang & Jung (2008), o espaço de funções admissíveis é composto pelo conjunto de todas as funções mensuráveis, definidas em no intervalo $[0,T]$, cujos valores estão entre $0$ e $0.9$. O limite $0.9$.

No nosso caso, vamos simplificar e restringir o espaço de funções admissíveis a funções da forma 

$$ 
u_{(a, b, c, t_1, t_2)}(t) = \begin{cases}
  a, & 0\leq t \leq t_1, \\
  b, & t_1 < t \leq t_2, \\
  c, & t_2 < t \leq T.
\end{cases}
$$

com as restrições

$$ 0 \leq t_1 < t_2 < T, \qquad 0\leq a, b, c \leq 0.9.
$$

## Projeto

1. Faça o ajuste dos parâmetros $\beta$ e $\gamma$ do modelos SIR clássico aos dados da epidemia de *Influenza* em um escola de meninos no norte da Inglaterra, com um total de 763 residentes, em um período de duas semanas, em que, a cada dia, o número de infectados é dado pela lista

$$ \text{Infectados} = [1, 3, 7, 25, 72, 222, 282, 256, 233, 189, 123, 70, 25, 11, 4]
$$

1. Usando os parâmetros $\beta$ e $\gamma$ obtidos no ajuste acima e usando $\nu=0.2$, encontre os parâmetros $a_*,b_*,c_*,{t_1}_*,{t_2}_*$, dentro das condições de admissibilidade, tais que a estratégia de vacinação $u_* = u_{(a_*,b_*,c_*,{t_1}_*,{t_2}_*)}$ minimiza o funcional-custo

$$ J(u) = \int_0^T \left( A_1 S(t) + A_2 I(t) + \frac{\tau}{2}u(t)^2\right) \;\rm dt,
$$

com $T=100$ (dias), $A_1 = 0.1$ (custo por indivíduo suscetível por dia), $A_2 = 1$ (custo por indivíduo infectado por dia), $\tau = 50$ (custo diário associado à estratégia de vacinação).


## Informações para a entrega do projeto:

- Enviar arquivo no formato de um **caderno Jupyter**.

- Enviar por correio eletrônico com cópia para Alejandro Cabrera e Ricardo Rosa.
    
- Enviar a mensagem com o assunto **Projeto 3 de Modelagem Matemática: Epidemiologia**.

- Nomear o arquivo do caderno Jupyter com o número do projeto e o seu nome, e.g. `Projeto3_Nome_do_Estudante.ipynb`.

- Imagens ou dados que precisem ser carregados pelo caderno devem ser enviados em um subdiretório, na mesma mensagem com o arquivo do caderno Jupyter.
    
- Podem enviar o caderno e o subdiretório agrupados em um único arquivo `.tar`, `.gz` ou semelhante.

## Organização do caderno:

O caderno Jupyter deve conter as seguintes informações:

- **Título:** "Projeto 3 de Modelagem Matemática: Epidemioloia"
- **Nome do estudante**
- **Data de entrega**
- **Objetivos**
- **Descrição do projeto**
- **Análise do problema** (incluindo dados trabalhados, métodos utilizadas, resultados obtidos, discussão dos resultados)

## Data de entrega

- Até a quarta-feira dia **22 de maio**.


---

### Apresentação em slides

Para visualizar na forma de slides, abrir um "terminal" *bash* e usar a linha de comando
```bash
jupyter nbconvert 12.05-Projeto3.ipynb --to slides --post serve
```

Se quiser adicionar a possibilidade de rolar a página, caso algum *slide* seja muito longo, incluir a opção
```bash
--SlidesExporter.reveal_scroll=True
```
Ou executar diretamente o código na célula a seguir.

In [1]:
!jupyter nbconvert 12.05-Projeto3.ipynb --to slides

[NbConvertApp] Converting notebook 12.05-Projeto3.ipynb to slides
[NbConvertApp] Writing 291068 bytes to 12.05-Projeto3.slides.html


<!--NAVIGATOR-->

---
[<- Aula 12: Epidemiologia - outros modelos compartimentais](12.00-Aula12.ipynb) | [Página Inicial](00-Sumario.ipynb) | [Aula 13: Epidemiologia: o modelo SIR em rede ->](13.00-Aula13.ipynb)

<a href="https://colab.research.google.com/github/rmsrosa/modelagem_matematica/blob/master/aulas/12.05-Projeto3.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<a href="https://mybinder.org/v2/gh/rmsrosa/modelagem_matematica/master?filepath=aulas/12.05-Projeto3.ipynb"><img align="left" src="https://mybinder.org/badge.svg" alt="Open in binder" title="Open and Execute in Binder"></a>

<a href="https://rmsrosa.github.io/modelagem_matematica/aulas/12.05-Projeto3.slides.html"><img align="left" src="https://rmsrosa.github.io/jupyterbookmaker/badges/slides_badge.svg" alt="Open slides" title="Open and View Slides"></a>
&nbsp;